In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import scipy.stats as stats

from sklearn import preprocessing
from sklearn import metrics
from sklearn import model_selection
from sklearn import feature_selection


from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import VotingRegressor



import warnings
warnings.filterwarnings('ignore')

In [7]:
df_time = pd.read_csv("C:/Piyush/Study/Great Learning/Hackathons/Files/Train Data/Train Data Zip/time_domain_features_train.csv")
df_freq = pd.read_csv("C:/Piyush/Study/Great Learning/Hackathons/Files/Train Data/Train Data Zip/frequency_domain_features_train.csv")
df_nlf = pd.read_csv("C:/Piyush/Study/Great Learning/Hackathons/Files/Train Data/Train Data Zip/heart_rate_non_linear_features_train.csv")

df_time_test = pd.read_csv('C:/Piyush/Study/Great Learning/Hackathons/Files/Test Data/Test Zip/time_domain_features_test.csv')
df_freq_test = pd.read_csv('C:/Piyush/Study/Great Learning/Hackathons/Files/Test Data/Test Zip/frequency_domain_features_test.csv')
df_nlf_test = pd.read_csv('C:/Piyush/Study/Great Learning/Hackathons/Files/Test Data/Test Zip/heart_rate_non_linear_features_test.csv')

df_test_y = pd.read_csv('C:\Piyush\Study\Great Learning\Hackathons\Files\SolutionFiles\RandomForestRegressor/0.018414562355024856__RandomForestRegressor__submission_0.2610179.csv')


frequency_domain_original_dataset = pd.read_csv("C:/Piyush/Study/Great Learning/Hackathons/Files/Train Data/Train Data Zip/frequency_domain_features_train.csv")
time_domain_original_dataset = pd.read_csv("C:/Piyush/Study/Great Learning/Hackathons/Files/Train Data/Train Data Zip/time_domain_features_train.csv")
non_linear_original_dataset = pd.read_csv("C:/Piyush/Study/Great Learning/Hackathons/Files/Train Data/Train Data Zip/heart_rate_non_linear_features_train.csv")

In [8]:
# Merging all the datasets into one single datasets
df = pd.concat([df_time, df_freq, df_nlf], axis=1, sort=False)
df_test = pd.concat([df_time_test, df_freq_test, df_nlf_test], axis=1, sort=False)

In [9]:
# Reordering the columns of the dataframe
df = df[['uuid', 'MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD', 'SDRR_RMSSD',
       'pNN25', 'pNN50', 'KURT', 'SKEW', 'MEAN_REL_RR', 'MEDIAN_REL_RR',
       'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDSD_REL_RR', 'SDRR_RMSSD_REL_RR',
       'KURT_REL_RR', 'SKEW_REL_RR', 'VLF', 'VLF_PCT', 'LF',
       'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU', 'TP', 'LF_HF', 'HF_LF',
       'SD1', 'SD2', 'sampen', 'higuci', 'condition', 'HR']]
df = df.iloc[:,2:]

df_test = df_test[['uuid', 'MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD', 'SDRR_RMSSD',
       'pNN25', 'pNN50', 'KURT', 'SKEW', 'MEAN_REL_RR', 'MEDIAN_REL_RR',
       'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDSD_REL_RR', 'SDRR_RMSSD_REL_RR',
       'KURT_REL_RR', 'SKEW_REL_RR', 'VLF', 'VLF_PCT', 'LF',
       'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU', 'TP', 'LF_HF', 'HF_LF',
       'SD1', 'SD2', 'sampen', 'higuci', 'condition']]
df_test = df_test.iloc[:,2:]

In [10]:
y_test = df_test_y[['HR']]

In [13]:
# Creating a new dataframe without the principal component dimensions
df_nPC = df[['MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD', 'SDRR_RMSSD',
       'pNN25', 'pNN50', 'KURT', 'SKEW', 'MEAN_REL_RR', 'MEDIAN_REL_RR',
       'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDSD_REL_RR', 'SDRR_RMSSD_REL_RR',
       'KURT_REL_RR', 'SKEW_REL_RR', 'VLF', 'LF', 'LF_NU',
       'HF', 'HF_NU', 'TP', 'LF_HF', 'HF_LF', 'SD1', 'SD2', 'sampen',
       'higuci', 'HR']]

df_test_nPC = df_test[['MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD', 'SDRR_RMSSD',
       'pNN25', 'pNN50', 'KURT', 'SKEW', 'MEAN_REL_RR', 'MEDIAN_REL_RR',
       'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDSD_REL_RR', 'SDRR_RMSSD_REL_RR',
       'KURT_REL_RR', 'SKEW_REL_RR', 'VLF', 'LF', 'LF_NU',
       'HF', 'HF_NU', 'TP', 'LF_HF', 'HF_LF', 'SD1', 'SD2', 'sampen',
       'higuci']]

In [14]:
pd.set_option('max_columns', None)

In [15]:
original_dataset = pd.merge(frequency_domain_original_dataset,time_domain_original_dataset,on='uuid',how='inner') 

In [16]:
original_dataset = pd.merge(original_dataset,non_linear_original_dataset,on='uuid',how='inner') 

In [17]:
col = original_dataset.pop("HR")
original_dataset.insert(36, col.name, col)

In [18]:
X = original_dataset.iloc[:,:-1]
y= original_dataset.iloc[:,-1:]

In [19]:
X.head()

,uuid,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,SD1,SD2,sampen,higuci,datasetId,condition
0,89df2855-56eb-4706-a23b-b39363dd605a,2661.894136,72.203287,1009.249419,27.375666,98.485263,15.522603,0.421047,1.514737,3686.666157,65.018055,0.015380,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,11.133333,0.533333,-0.856554,0.335218,-0.000203,-0.000179,0.017080,0.007969,0.007969,2.143342,-0.856554,0.335218,11.001565,199.061782,2.139754,1.163485,2,no stress
1,80c795e4-aa56-4cc0-939c-19634b89cbb2,2314.265450,76.975728,690.113275,22.954139,99.695397,2.108525,0.070133,0.304603,3006.487251,327.296635,0.003055,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,5.600000,0.000000,-0.408190,-0.155286,-0.000059,0.000611,0.013978,0.004769,0.004769,2.930855,-0.408190,-0.155286,9.170129,114.634458,2.174499,1.084711,2,interruption
2,c2d5d102-967c-487d-88f2-8b005a449f3e,1373.887112,51.152225,1298.222619,48.335104,98.950472,13.769729,0.512671,1.049528,2685.879461,94.280910,0.010607,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,13.066667,0.200000,0.351789,-0.656813,-0.000011,-0.000263,0.018539,0.008716,0.008716,2.127053,0.351789,-0.656813,11.533417,118.939253,2.135350,1.176315,2,interruption
3,37eabc44-1349-4040-8896-0d113ad4811f,2410.357408,70.180308,1005.981659,29.290305,98.224706,18.181913,0.529387,1.775294,3434.520980,55.328701,0.018074,881.757865,893.460030,90.370537,15.720468,15.720068,5.748591,11.800000,0.133333,-0.504947,-0.386138,0.000112,0.000494,0.017761,0.008660,0.008660,2.050988,-0.504947,-0.386138,11.119476,127.318597,2.178341,1.179688,2,no stress
4,aa777a6a-7aa3-4f6e-aced-70f8691dd2b7,1151.177330,43.918366,1421.782051,54.242160,96.720007,48.215822,1.839473,3.279993,2621.175204,29.487873,0.033912,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,20.200000,0.200000,-0.548408,-0.154252,-0.000100,-0.002736,0.023715,0.013055,0.013055,1.816544,-0.548408,-0.154252,13.590641,87.718281,2.221121,1.249612,2,no stress


In [20]:
y.head()

,HR
0,69.499952
1,64.363150
2,67.450066
3,68.809562
4,74.565728


In [21]:
X['condition'].value_counts()

no stress        200082
interruption     105150
time pressure     64057
Name: condition, dtype: int64

In [22]:
X_trainset,X_testset,y_trainset,y_testset = model_selection.train_test_split(X,y,test_size=0.3,random_state=1)

In [23]:
X_trainset.drop(['uuid','datasetId'],axis=1,inplace=True)

In [24]:
X_trainset['condition'] =X_trainset['condition'].map({'no stress':0, 'interruption':1,'time pressure':2 })

In [25]:
X_testset.drop(['uuid','datasetId'],axis=1,inplace=True)

In [26]:
X_testset['condition'] =X_testset['condition'].map({'no stress':0, 'interruption':1,'time pressure':2 })

In [27]:
X_trainset['condition'].value_counts()

0    139963
1     73689
2     44850
Name: condition, dtype: int64

In [28]:
X_testset['condition'].value_counts()

0    60119
1    31461
2    19207
Name: condition, dtype: int64

In [29]:
X_trainset.head()

,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,SD1,SD2,sampen,higuci,condition
263844,919.016936,32.821653,1825.815553,65.206943,97.065419,55.199930,1.971403,2.934581,2800.032418,33.076411,0.030233,837.355539,843.010715,72.527620,21.458314,21.458312,3.379931,25.466667,0.866667,-0.339559,-0.364001,0.000006,-0.002244,0.025766,0.013806,0.013806,1.866213,-0.339559,-0.364001,15.178382,101.440268,2.201144,1.249238,0
304442,573.858535,42.882651,749.740376,56.025750,98.088848,14.607855,1.091599,1.911152,1338.206766,51.324466,0.019484,777.424227,776.059315,49.694040,11.143164,11.143069,4.459599,1.933333,0.000000,-0.258963,0.129651,-0.000060,-0.000890,0.014308,0.006307,0.006307,2.268600,-0.258963,0.129651,7.881969,69.834588,2.195039,1.135493,0
64629,2942.069867,66.666656,1467.477007,33.252706,99.758088,3.558618,0.080638,0.241912,4413.105492,412.372716,0.002425,1002.621602,1020.694950,128.303615,18.942952,18.941474,6.773159,20.733333,0.666667,0.531892,-0.366677,0.000287,-0.000725,0.018538,0.008090,0.008090,2.291515,0.531892,-0.366677,13.398114,180.953380,2.060168,1.168586,2
130476,457.014323,40.525838,566.325564,50.219035,84.438407,104.371080,9.255127,15.561593,1127.710968,5.426077,0.184295,746.681177,748.080510,45.300162,14.763308,14.763266,3.068429,8.600000,0.066667,-0.349695,-0.250273,-0.000056,-0.000615,0.020213,0.013954,0.013954,1.448556,-0.349695,-0.250273,10.442689,63.207275,2.203743,1.337419,1
364962,832.688712,46.863734,933.222095,52.521754,98.843516,10.918838,0.614512,1.156484,1776.829645,85.468996,0.011700,837.551625,837.341680,61.729762,13.359126,13.358973,4.620793,5.133333,0.133333,-0.090814,-0.143324,-0.000075,-0.001411,0.016127,0.006911,0.006911,2.333311,-0.090814,-0.143324,9.449373,86.786153,2.187653,1.145672,1


In [30]:
X_testset.head()

,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,SD1,SD2,sampen,higuci,condition
47340,2240.309125,71.772678,862.814581,27.641950,97.926221,18.271780,0.585372,2.073779,3121.395486,47.221157,0.021177,896.259119,890.954330,107.500849,14.470598,14.470382,7.428916,8.200000,0.400000,-0.504123,0.159592,0.000075,-0.000616,0.016426,0.007759,0.007759,2.117102,-0.504123,0.159592,10.235520,151.684209,2.204324,1.162558,2
93858,410.649097,36.768573,689.838660,61.766562,97.683327,16.360321,1.464865,2.316673,1116.848078,42.165349,0.023716,782.200972,784.207490,44.505190,10.956871,10.956847,4.061852,2.600000,0.000000,0.005592,-0.217033,0.000029,-0.000741,0.014025,0.006607,0.006607,2.122641,0.005592,-0.217033,7.750247,62.460848,2.169347,1.157278,0
201163,1733.538326,71.269718,680.589843,27.980602,97.390628,18.234939,0.749680,2.609372,2432.363108,37.323395,0.026793,801.311974,803.009105,100.133280,12.269379,12.269321,8.161234,4.133333,0.000000,0.214343,0.122436,0.000054,0.000052,0.014900,0.007326,0.007326,2.033823,0.214343,0.122436,8.678615,141.343656,2.107053,1.184945,1
364634,1782.496592,58.032632,1247.073484,40.600895,96.743964,41.971774,1.366472,3.256036,3071.541851,29.712194,0.033656,812.978489,817.475120,88.547464,16.963464,16.963211,5.219893,10.066667,1.466667,-0.049779,-0.274960,0.000117,0.000126,0.022035,0.010894,0.010894,2.022644,-0.049779,-0.274960,11.998805,124.648849,2.177006,1.147558,0
54177,1431.550448,56.058137,832.728144,32.608832,74.209035,289.410343,11.333030,25.790965,2553.688934,2.877327,0.347545,669.485151,689.196165,96.627965,19.707569,19.707175,4.903089,12.000000,2.933333,1.460162,-1.196760,-0.000166,-0.001624,0.033459,0.024741,0.024741,1.352399,1.460162,-1.196760,13.939728,135.939733,1.914554,1.321684,0


In [31]:
%%time

power = preprocessing.PowerTransformer(method='yeo-johnson', standardize=True)
df_transformed = power.fit_transform(X_trainset)
X_trainset_transformed = pd.DataFrame(df_transformed, columns = X_trainset.columns)

Wall time: 14.9 s


In [32]:
%%time

df_test_transformed = power.transform(X_testset)
X_testset_transformed = pd.DataFrame(df_test_transformed, columns = X_testset.columns)

Wall time: 211 ms


In [33]:
def print_estimator_name(estimator):
    return str(estimator.__class__.__name__)

## GradientBoost   

In [ ]:
%%time

gradientBoost = GradientBoostingRegressor( n_estimators=50,loss = 'huber',criterion = 'mae' ,random_state=1)
gradientBoost.fit(X_trainset_transformed,y_trainset)
GB_y_predicted = gradientBoost.predict(X_testset_transformed)

In [ ]:
%%time
train_score  = np.round(gradientBoost.score(X_trainset_transformed,y_trainset),2)
test_score   = np.round(gradientBoost.score(X_testset_transformed,y_testset),2)
mae          = np.round(metrics.mean_absolute_error(y_trainset,GB_y_predicted),7)
submissionscore = np.round(gradientBoost.score(GB_y_predicted,y_test),2)
submissionMAE = np.round(metrics.mean_absolute_error(GB_y_predicted,y_test),7)
print("Train Accuracy ",train_score)
print("Test Accuracy ",test_score)
print("Mean Absolute Error GradientBoost",mae)
print("Submission score GradientBoost",submissionscore)
print("Mean Absolute Error submission GradientBoost",submissionMAE)

In [ ]:
df_f = pd.DataFrame(GB_y_predicted)
df_sub = pd.concat([df_nlf_test['uuid'], df_f], axis=1, sort=False)
df_sub.columns = ['uuid', 'HR']
df_sub.to_csv('C:/Piyush/Study/Great Learning/Hackathons/Files/SolutionFiles/17122020'+ print_estimator_name(gradientBoost)+'_'+str(submissionMAE)+'_'+str(submissionscore)+'_' + 'submission.csv')

## Random Forest with MAE

In [ ]:
%%time

RF_100 = RandomForestRegressor(n_estimators=100,criterion='mae',random_state = 1,n_jobs=-1)
RF_100.fit(X_trainset_transformed,y_trainset)
RF_100_y_predicted = RF_100.predict(X_testset_transformed)

In [ ]:
%%time

# RF with 100 Estimators
train_score  = np.round(RF_100.score(X_trainset_transformed,y_trainset),2)
test_score   = np.round(RF_100.score(X_testset_transformed,y_testset),2)
mae          = np.round(metrics.mean_absolute_error(y_testset,RF_100_y_predicted),7)
submissionscore = np.round(gradientBoost.score(RF_100_y_predicted,y_test),2)
submissionMAE = np.round(metrics.mean_absolute_error(RF_100_y_predicted,y_test),7)
print("Train Accuracy ",train_score)
print("Test Accuracy ",test_score)
print("Mean Absolute Error Random Forest",mae)
print("Submission score Random Forest",submissionscore)
print("Mean Absolute Error submission Random Forest",submissionMAE)

In [ ]:
df_f = pd.DataFrame(RF_100_y_predicted)
df_sub = pd.concat([df_nlf_test['uuid'], df_f], axis=1, sort=False)
df_sub.columns = ['uuid', 'HR']
df_sub.to_csv('C:/Piyush/Study/Great Learning/Hackathons/Files/SolutionFiles/17122020'+ print_estimator_name(gradientBoost)+'_'+str(submissionMAE)+'_'+str(submissionscore)+'_' + 'submission.csv')

## Random Forest with MSE 

In [ ]:
%%time
RF_100 = RandomForestRegressor(n_estimators=100,criterion='mse',random_state = 1,n_jobs=-1)
RF_100.fit(X_trainset_transformed,y_trainset)
RF_100_y_predicted = RF_100.predict(X_testset_transformed)

In [ ]:
%%time

# RF with 100 Estimators
train_score  = np.round(RF_100.score(X_trainset_transformed,y_trainset),2)
test_score   = np.round(RF_100.score(X_testset_transformed,y_testset),2)
mae          = np.round(metrics.mean_absolute_error(y_testset,RF_100_y_predicted),7)
submissionscore = np.round(gradientBoost.score(RF_100_y_predicted,y_test),2)
submissionMAE = np.round(metrics.mean_absolute_error(RF_100_y_predicted,y_test),7)
print("Train Accuracy ",train_score)
print("Test Accuracy ",test_score)
print("Mean Absolute Error Random Forest",mae)
print("Submission score Random Forest",submissionscore)
print("Mean Absolute Error submission Random Forest",submissionMAE)

In [ ]:
df_f = pd.DataFrame(RF_100_y_predicted)
df_sub = pd.concat([df_nlf_test['uuid'], df_f], axis=1, sort=False)
df_sub.columns = ['uuid', 'HR']
df_sub.to_csv('C:/Piyush/Study/Great Learning/Hackathons/Files/SolutionFiles/17122020'+ print_estimator_name(gradientBoost)+'_'+str(submissionMAE)+'_'+str(submissionscore)+'_' + 'submission.csv')

## AdaBoost Using RF with 100 Estimators

In [ ]:
%%time
# AdaBoost Using RF with 100 Estimators

AdaBoostReg = AdaBoostRegressor(base_estimator = RF_100, random_state=1, n_estimators=10)
AdaBoostReg.fit(X_trainset_transformed,y_trainset)
AdaBoostReg_y_predicted = AdaBoostReg.predict(X_testset_transformed)

In [ ]:
%%time

# Adaboost using RF with 100 Estimators
train_score  = np.round(AdaBoostReg.score(X_trainset_transformed,y_trainset),2)
test_score   = np.round(AdaBoostReg.score(X_testset_transformed,y_testset),2)
mae          = np.round(metrics.mean_absolute_error(y_testset,AdaBoostReg_y_predicted),7)
submissionscore = np.round(gradientBoost.score(AdaBoostReg_y_predicted,y_test),2)
submissionMAE = np.round(metrics.mean_absolute_error(AdaBoostReg_y_predicted,y_test),7)
print("Train Accuracy ",train_score)
print("Test Accuracy ",test_score)
print("Mean Absolute Error Adaboost ",mae)
print("Submission score Random Forest",submissionscore)
print("Mean Absolute Error submission Random Forest",submissionMAE)

In [ ]:
df_f = pd.DataFrame(AdaBoostReg_y_predicted)
df_sub = pd.concat([df_nlf_test['uuid'], df_f], axis=1, sort=False)
df_sub.columns = ['uuid', 'HR']
df_sub.to_csv('C:/Piyush/Study/Great Learning/Hackathons/Files/SolutionFiles/17122020'+ print_estimator_name(gradientBoost)+'_'+str(submissionMAE)+'_'+str(submis